In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
path = "C:/Users/STEVEN H/Desktop/5420 anomaly detection/2.  Feature Engineering Credit card Transaction Data/5420 a2/" 
df = pd.read_csv(path + 'purchase_credit_card.csv') #, encoding = "ISO-8859-1")    
df.head(5)  

,Year-Month,Agency Number,Agency Name,Cardholder Last Name,Cardholder First Initial,Description,Amount,Vendor,Transaction Date,Posted Date,Merchant Category Code (MCC)
0,201307,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,GENERAL PURCHASE,890.00,NACAS,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,CHARITABLE AND SOCIAL SERVICE ORGANIZATIONS
1,201307,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,ROOM CHARGES,368.96,SHERATON HOTEL,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,SHERATON
2,201307,1000,OKLAHOMA STATE UNIVERSITY,Massey,J,GENERAL PURCHASE,165.82,SEARS.COM 9300,07/29/2013 12:00:00 AM,07/31/2013 12:00:00 AM,DIRCT MARKETING/DIRCT MARKETERS--NOT ELSEWHERE...
3,201307,1000,OKLAHOMA STATE UNIVERSITY,Massey,T,GENERAL PURCHASE,96.39,WAL-MART #0137,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,"GROCERY STORES,AND SUPERMARKETS"
4,201307,1000,OKLAHOMA STATE UNIVERSITY,Mauro-Herrera,M,HAMMERMILL COPY PLUS COPY EA,125.96,STAPLES DIRECT,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,"STATIONERY, OFFICE SUPPLIES, PRINTING AND WRIT..."


## Section 1 Data Preparation

In [3]:
# check dimensions of the dataset, we found it has 442,458 rows and 11 columns 
print(df.shape)
print(df.columns) # check column names
df.describe() # Get the Simple Summary Statistics 

(442458, 11)
Index(['Year-Month', 'Agency Number', 'Agency Name', 'Cardholder Last Name',
       'Cardholder First Initial', 'Description', 'Amount', 'Vendor',
       'Transaction Date', 'Posted Date', 'Merchant Category Code (MCC)'],
      dtype='object')


,Year-Month,Agency Number,Amount
count,442458.000000,442458.000000,4.424580e+05
mean,201357.284375,42785.860353,4.249912e+02
std,47.107417,33378.461293,5.266509e+03
min,201307.000000,1000.000000,-4.286304e+04
25%,201309.000000,1000.000000,3.091000e+01
50%,201401.000000,47700.000000,1.048900e+02
75%,201404.000000,76000.000000,3.450000e+02
max,201406.000000,98000.000000,1.903858e+06


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442458 entries, 0 to 442457
Data columns (total 11 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Year-Month                    442458 non-null  int64  
 1   Agency Number                 442458 non-null  int64  
 2   Agency Name                   442458 non-null  object 
 3   Cardholder Last Name          442458 non-null  object 
 4   Cardholder First Initial      442458 non-null  object 
 5   Description                   442458 non-null  object 
 6   Amount                        442458 non-null  float64
 7   Vendor                        442458 non-null  object 
 8   Transaction Date              442458 non-null  object 
 9   Posted Date                   442458 non-null  object 
 10  Merchant Category Code (MCC)  442458 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 37.1+ MB


## Section 2 Feature Engineering

1. Post date-transaction date

In [5]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
df['Posted Date'] = pd.to_datetime(df['Posted Date'])

Date_Diff, represents the time difference in days between the transaction date and the posted date. It can provide insights into how quickly transactions are processed or if there are any delays in posting. Thus, this feature do have benefits for the business

2.Average amounts spent per transaction in the past one week

In [6]:
df = df.sort_values(by='Transaction Date')

def rolling_amount(df, window, agg_func):
    return df.rolling(window, on='Transaction Date')['Amount'].agg(agg_func)

df['Avg_Amount_7_days'] = df.groupby('Cardholder Last Name').apply(rolling_amount, '7D', 'mean').reset_index(0, drop=True)

By comparing the current transaction amount with the rolling 7-day average, I can identify transactions that deviate significantly from a cardholder's average spending behavior. 

3.Average amounts spent per day in the past one week

In [7]:
daily_df = df.resample('D', on='Transaction Date').sum().reset_index()
daily_df['AvgAmount_perday_7days'] = daily_df['Amount'].rolling(7).mean()

This kind of feature can assist in budgeting and monitoring cardholder expenses. It provides an average benchmark to compare individual transactions against the group ones.

4.Average amount per day spent over three days on all transactions up to this one on the same merchant type as this transaction

In [8]:
df['Avg_Amount_3_days_MCC'] = df.groupby('Merchant Category Code (MCC)').apply(rolling_amount, '3D', 'mean').reset_index(0, drop=True)

We can use this feature to track some bad amounts within 3 days,it helps identify spending patterns based on specific days of the week.

5.Transaction Day of the Week: The day of the week when the transaction occurred 

In [9]:
# Assuming 'Transaction Date' column contains datetime values
df['Transaction Day of the Week'] = df['Transaction Date'].dt.dayofweek

# Map day of the week numeric values to corresponding day names
day_mapping = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
df['Transaction Day of the Week'] = df['Transaction Day of the Week'].map(day_mapping)

It facilitates the identification of anomalies by detecting unexpected spending patterns on specific days of the week.

6.Total number of transactions on the same day up to this transaction

In [10]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
date_counts = df['Transaction Date'].value_counts()
df['Total_Transactions_Same_Day'] = df['Transaction Date'].map(date_counts)

It can identifies anomalies in transaction density and unusual spikes in activity.

7.Total amount spent on the same day up to this transaction

In [11]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
df['Total_Amount_Same_Day'] = df.groupby(df['Transaction Date'].dt.date)['Amount'].cumsum()

We can use this feature to highlights anomalies in cumulative spending amount, indicating unusual spending behavior.

8. Transaction Season

In [12]:
df['Year-Month'] = df['Year-Month'].astype(str)
df['Year'] = df['Year-Month'].str[:4].astype(int)
df['Month'] = df['Year-Month'].str[4:].astype(int)

season_mapping = {1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring', 6: 'Summer',
                  7: 'Summer', 8: 'Summer', 9: 'Fall', 10: 'Fall', 11: 'Fall', 12: 'Winter'}
df['Season'] = df['Month'].map(season_mapping)


It can detects anomalies by comparing spending patterns with expected seasonal variations.

9. Transaction Amount Deviation from Daily Average

In [13]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
df['Daily_Avg_Amount'] = df.groupby(df['Transaction Date'].dt.date)['Amount'].transform('mean')
df['Amount_Deviation_Daily'] = df['Amount']- df['Daily_Avg_Amount']

I can tracking the average spending per day on a specific merchant type to detect the number which is too low or too high

10.Transaction Amount Deviation from Merchant Category Average

In [14]:
df['Merchant_Category_Avg_Amount'] = df.groupby('Merchant Category Code (MCC)')['Amount'].transform('mean')
df['Amount_Deviation_Merchant'] = df['Amount'] -df['Merchant_Category_Avg_Amount']

Helps identify big or small deviations in spending patterns compared to the average daily spending.

11.Time Difference between Current and Previous Transaction

In [15]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
df['Previous_Transaction_Date'] = df.groupby('Cardholder Last Name')['Transaction Date'].shift()
df['Time_Difference_Prev_Trans'] = (df['Transaction Date'] -df['Previous_Transaction_Date']).dt.total_seconds()/60

Find spending variations within specific merchant categories, to classify the different one

12.Frequency of Transactions within Merchant Category

In [16]:
df['Merchant_Category_Frequency'] = df.groupby(['Cardholder Last Name', 'Merchant Category Code (MCC)'])['Amount'].transform('count')

this features can identifies patterns and anomalies in usual transaction timing.

13.Transaction Amount Relative to Cardholder's Average

In [17]:
df['Cardholder_Avg_Amount'] = df.groupby(['Cardholder Last Name'])['Amount'].transform('mean')
df['Amount_Relative_Cardholder'] = df['Amount'] / df['Cardholder_Avg_Amount']

Indicates the wrong cardholder's engagement within specific merchant categories.

14.Transaction Amount Deviation from Monthly Average

In [18]:
# change the type first 
df['Year-Month'] = df['Year-Month'].astype(str)
df['Monthly_Avg_Amount'] = df.groupby('Year-Month')['Amount'].transform('mean')
df['Amount_Deviation_Monthly'] = df['Amount'] - df['Monthly_Avg_Amount']

Captures unsual variations in spending compared to the average monthly spending.

15.Maximum Amount Spent on the Same Day up to the Current Transaction

In [19]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
df['Max_Amount_Same_Day'] = df.groupby(df['Transaction Date'].dt.date)['Amount'].cummax()

 Reflects recent transaction activity and engagement levels to detect the unnormal one.

In [20]:
pd.set_option('display.max_columns', None)  # Set the option to display all columns
print(df)

       Year-Month  Agency Number                             Agency Name  \
31364      201310           1000               OKLAHOMA STATE UNIVERSITY   
147048     201307          63200   SPEECH-LANGUAGE PATHOLOGY & AUDIOLOGY   
9276       201307           1000               OKLAHOMA STATE UNIVERSITY   
1901       201307           1000               OKLAHOMA STATE UNIVERSITY   
3487       201307           1000               OKLAHOMA STATE UNIVERSITY   
...           ...            ...                                     ...   
437420     201406          77000   UNIV. OF OKLA. HEALTH SCIENCES CENTER   
405350     201406          48000               N. E. OKLA. A & M COLLEGE   
437481     201406          77000   UNIV. OF OKLA. HEALTH SCIENCES CENTER   
385939     201406           1000               OKLAHOMA STATE UNIVERSITY   
410511     201406          61000  REGIONAL UNIVERSITY SYSTEM OF OKLAHOMA   

       Cardholder Last Name Cardholder First Initial  \
31364                Tucker    

After preprocessing the dataset and engineering relevant features, I can conclude that generate multiple features in different ways are very useful for detecting anomalies in cardholder spending behavior. For example, if a cardholder's current transaction amount deviates significantly from their average spending in the past week, it could signal potential fraudulent activity or unusual spending patterns. By monitoring and analyzing these deviations, we can identify suspicious transactions and take appropriate action to mitigate risks and protect their customers. This features can be used for unsupervised learning models and really do benefits to the accuracy of the models.

In [21]:
df.describe()

,Agency Number,Amount,Avg_Amount_7_days,Avg_Amount_3_days_MCC,Total_Transactions_Same_Day,Total_Amount_Same_Day,Year,Month,Daily_Avg_Amount,Amount_Deviation_Daily,Merchant_Category_Avg_Amount,Amount_Deviation_Merchant,Time_Difference_Prev_Trans,Merchant_Category_Frequency,Cardholder_Avg_Amount,Amount_Relative_Cardholder,Monthly_Avg_Amount,Amount_Deviation_Monthly,Max_Amount_Same_Day
count,442458.000000,4.424580e+05,4.424580e+05,442458.000000,442458.000000,4.424580e+05,442458.000000,442458.000000,442458.000000,4.424580e+05,442458.000000,4.424580e+05,438547.000000,442458.000000,442458.000000,4.424580e+05,442458.000000,4.424580e+05,4.424580e+05
mean,42785.860353,4.249912e+02,4.289410e+02,424.765577,1543.941807,3.354251e+05,2013.509058,6.378526,424.991170,2.423494e-15,424.991170,-9.718642e-15,3521.849380,198.506538,424.991170,NaN,424.991170,-1.234152e-14,4.545056e+04
std,33378.461293,5.266509e+03,4.111175e+03,2491.255370,425.345374,2.709991e+05,0.499919,3.359571,535.130739,5.239251e+03,378.726005,5.252874e+03,11742.129903,607.170870,2415.589995,NaN,25.290594,5.266448e+03,1.363119e+05
min,1000.000000,-4.286304e+04,-2.148950e+04,-11757.590000,1.000000,-2.199383e+04,2013.000000,1.000000,-3047.263333,-4.337078e+04,-243.007333,-4.305414e+04,0.000000,1.000000,-112.015000,-inf,380.471545,-4.332389e+04,-2.611940e+03
25%,1000.000000,3.091000e+01,1.092602e+02,136.496688,1419.000000,1.234505e+05,2013.000000,3.000000,361.428038,-3.744098e+02,191.096308,-3.651786e+02,0.000000,7.000000,196.581582,1.265425e-01,405.716869,-3.877816e+02,8.446970e+03
50%,47700.000000,1.048900e+02,2.349029e+02,290.644971,1686.000000,2.990887e+05,2014.000000,6.000000,402.635719,-2.818232e+02,410.479644,-1.405419e+02,0.000000,24.000000,289.078843,4.063248e-01,435.781619,-3.148565e+02,1.915571e+04
75%,76000.000000,3.450000e+02,4.084201e+02,490.283028,1820.000000,4.969065e+05,2014.000000,9.000000,453.814693,-5.921201e+01,529.877561,-7.923427e+00,2880.000000,84.000000,412.368695,1.143629e+00,451.768880,-7.952813e+01,4.204754e+04
max,98000.000000,1.903858e+06,1.903858e+06,951942.550000,2122.000000,2.648101e+06,2014.000000,12.000000,343148.500000,1.902342e+06,4823.344647,1.900552e+06,528480.000000,4481.000000,609039.725000,inf,460.854807,1.903407e+06,1.903858e+06


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 442458 entries, 31364 to 410511
Data columns (total 31 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Year-Month                    442458 non-null  object        
 1   Agency Number                 442458 non-null  int64         
 2   Agency Name                   442458 non-null  object        
 3   Cardholder Last Name          442458 non-null  object        
 4   Cardholder First Initial      442458 non-null  object        
 5   Description                   442458 non-null  object        
 6   Amount                        442458 non-null  float64       
 7   Vendor                        442458 non-null  object        
 8   Transaction Date              442458 non-null  datetime64[ns]
 9   Posted Date                   442458 non-null  datetime64[ns]
 10  Merchant Category Code (MCC)  442458 non-null  object        
 11  Avg_Amoun

## Section 3 Modeling

In [23]:
import pandas as pd

# Binning 'Amount' column
num_bins = 5  # Specify the number of bins
df['Amount_Binned'] = pd.cut(df['Amount'], bins=num_bins, labels=False)

# Binning 'Avg_Amount_7_days' column
custom_bins = [0, 100, 500, 1000, float('inf')]  # Define custom bin edges
df['Avg_Amount_7_days_Binned'] = pd.cut(df['Avg_Amount_7_days'], bins=custom_bins, labels=False)



## 0. Generate New Features

In [24]:
# Compute statistical features
df['Amount_Mean'] = df.groupby(['Vendor'])['Amount'].transform('mean')
df['Amount_Std'] = df.groupby(['Vendor'])['Amount'].transform('std')
df['Total_Amount_Same_Day_Min'] = df.groupby(['Vendor'])['Total_Amount_Same_Day'].transform('min')
df['Total_Amount_Same_Day_Max'] = df.groupby(['Vendor'])['Total_Amount_Same_Day'].transform('max')


### a. Data Generation

In [26]:
# contamination = 0.05
# n_train = 500
# n_test = 500
# n_features = 6

# X_train, X_test, y_train, y_test = generate_data(
#     n_train=n_train,
#     n_test=n_test,
#     n_features=n_features,
#     contamination=contamination,
#     random_state=123
# )


The first section of the code is used for data generation. The generate_data function from PyOD's utility is used to generate random data for training and testing. The number of samples in training and test data is 500 each, with 6 features. The contamination rate is set to 0.05, meaning that approximately 5% of the total data points are outliers.

### b. Model Training and Prediction

In [28]:
from pyod.models.hbos import HBOS
hbos = HBOS(contamination=contamination)
hbos.fit(X_train)

y_train_scores = hbos.decision_function(X_train)
y_train_pred = hbos.predict(X_train)

y_test_scores = hbos.decision_function(X_test)
y_test_pred = hbos.predict(X_test)


C:\Users\STEVEN H\Anaconda3\envs\week-1\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):


NameError: name 'X_train' is not defined

In this part, we instantiate and fit the HBOS (Histogram-Based Outlier Score) model, a fast unsupervised anomaly detector. The contamination parameter is used to define the proportion of outliers in the data. The predict function is used to predict if a particular sample is an outlier or not. The decision_function calculates the anomaly score for each observation.

### c. Statistics and Visualization

In [ ]:
threshold = hbos.threshold_
print("The threshold for the defined contamination rate:", threshold)
print("Training data:", count_stat(y_train_pred))
print("Test data:", count_stat(y_test_pred))
plt.hist(y_test_scores, bins='auto')
plt.title("Histogram of Anomaly Scores")
plt.xlabel("HBOS")
plt.show()
descriptive_stat = descriptive_stat_threshold(X_train, y_train_scores, threshold)
print(descriptive_stat)


The very low values between 11 and 17 suggest that there are very few data points that the model considers to be mildly anomalous. The low values between 7 and 11 indicate that even fewer data points are considered strongly anomalous.

This information can be used to set a threshold for determining which points to consider as outliers. We can might decide to label all points with a score from 7-11 as outliers, as they represent the top end of the anomaly score distribution.

### d. Confusion Matrix

In [ ]:
cm_train = confusion_matrix(y_train, y_train_scores, threshold)
print("Confusion matrix for training data:")
print(cm_train)


### e. Hyperparameters Testing

In [ ]:
from sklearn.metrics import f1_score

# Define the range of contamination values
contamination_values = np.linspace(0.01, 0.1, 10)

# Initialize lists to store the results
f1_scores_train = []
f1_scores_test = []

# Loop over the contamination values
for contamination in contamination_values:
    # Fit the HBOS model
    hbos = HBOS(contamination=contamination)
    hbos.fit(X_train)
    
    # Predict the labels for the training data and compute the F1-score
    y_train_pred = hbos.predict(X_train)
    f1_train = f1_score(y_train, y_train_pred)
    f1_scores_train.append(f1_train)
    
    # Predict the labels for the test data and compute the F1-score
    y_test_pred = hbos.predict(X_test)
    f1_test = f1_score(y_test, y_test_pred)
    f1_scores_test.append(f1_test)

# Plot the F1-scores as a function of the contamination parameter
plt.figure(figsize=(10, 6))
plt.plot(contamination_values, f1_scores_train, label='Train')
plt.plot(contamination_values, f1_scores_test, label='Test')
plt.xlabel('Contamination')
plt.ylabel('F1-score')
plt.legend()
plt.title('Hyperparameter Tuning of HBOS Contamination Parameter')
plt.grid(True)
plt.show()


When contamination is between 0 and 0.05, both the training and test scores are improving. This suggests that the model is learning the underlying patterns in the data and generalizing well to unseen data.

Thus, in this case, the optimal contamination parameter appears to be around 0.05, where the test F1-score is at its peak. It's essential to remember that the exact value may vary depending on the randomness in the data split and other factors. Therefore, it's good practice to perform multiple runs or use cross-validation to confirm the optimal parameter.

## ECODS

### 1. Data Generation and Histogram Plotting

The histogram plotted provides a visual representation of the frequency of data points in different ranges. 

In [ ]:
# Generate a distribution
np.random.seed(123)
shape, scale = 10, 2
s1 = np.random.gamma(shape, scale, 1000)
s2 = np.random.gamma(shape * 2, scale * 2, 1000)
s3 = np.random.normal(loc=0, scale=5, size=1000)
sample = np.hstack((s1, s2, s3))

# Plot the histogram
plt.hist(sample, bins=50)
plt.title('Histogram')
plt.show()


### 2. Empirical Cumulative Distribution Function (CDF) Plotting

In [ ]:
# Fit a CDF
sample_ecdf = ECDF(sample)

# Plot the CDF
plt.plot(sample_ecdf.x, sample_ecdf.y)
plt.title('Empirical CDF')
plt.show()

# Calculate probabilities using the ECDF
probabilities = [-20, -2, 0, 25, 50, 75, 100, 125, 140, 150]
for p in probabilities:
    print('P(x < {}): {:.4f}'.format(p, sample_ecdf(p)))


Following the histogram, I fit an Empirical Cumulative Distribution Function (ECDF) to the sample data. The ECDF plot reveals how the probabilities are distributed across the entire sample. With the printed probabilities, it's evident that the majority of the data points fall between 25 and 100.

### 3. Synthetic Dataset Generation and Scatter Plotting

In [ ]:
# Generate synthetic dataset
contamination = 0.05  # percentage of outliers
n_train = 350000      # number of training points
n_test = 92458        # number of testing points
X_train, X_test, _, _ = generate_data(
    n_train=n_train,
    n_test=n_test,
    n_features=df.shape[1],
    contamination=contamination,
    random_state=123
)
X_train_pd = pd.DataFrame(X_train)

# Plot the scatter plot of the first two features
plt.scatter(X_train_pd.iloc[:, 0], X_train_pd.iloc[:, 1], alpha=0.8)
plt.title('Scatter Plot')
plt.xlabel('Feature 0')
plt.ylabel('Feature 1')
plt.show()


The scatter plot of the first two features provides a visual understanding of the data spread and potentially identify regions where outliers may exist.

### 4. Model Fitting and Predicting with ECOD

In [ ]:
# Fit ECOD model
ecod = ECOD(contamination=contamination)
ecod.fit(X_train)

# Training data
y_train_scores = ecod.decision_function(X_train)
y_train_pred = ecod.predict(X_train)

# Test data
X_test_pd = pd.DataFrame(X_test)
y_test_scores = ecod.decision_function(X_test)
y_test_pred = ecod.predict(X_test)

# Count statistics
print("Training Data:", count_stat(y_train_pred))
print("Testing Data:", count_stat(y_test_pred))


To find outliers within this synthetic dataset, I employed the Ensemble of Covariance-based Outlier Detectors (ECOD) model. After fitting the model, it was used to predict both on the training and testing data, providing an initial assessment of how well the model can identify outliers. With the training data, the ECOD model identified 17,500 data points as outliers (5% of the data), which matches the contamination factor I had initially set, suggesting a good fit for the model.

### 5. Outlier Score Distribution

In [ ]:
# Histogram of outlier scores
plt.hist(y_train_scores, bins='auto')
plt.title('Outlier Score Distribution')
plt.show()


The histogram of these scores displays the distribution of outlier scores. The varying range of scores suggests that the outliers are spread across different regions of the data space. The outliers mainly assembled between 40 80, a small number between 120-160.

### 6. Descriptive Statistics Based on Threshold

In [ ]:
# Descriptive statistics based on the threshold
def descriptive_stat_threshold(df, pred_score, threshold):
    df = pd.DataFrame(df)
    df['Anomaly_Score'] = pred_score
    df['Group'] = np.where(df['Anomaly_Score'] < threshold, 'Normal', 'Outlier')
    cnt = df.groupby('Group')['Anomaly_Score'].count().reset_index().rename(columns={'Anomaly_Score': 'Count'})
    cnt['Count %'] = (cnt['Count'] / cnt['Count'].sum()) * 100
    stat = df.groupby('Group').mean().round(2).reset_index()
    stat = cnt.merge(stat, left_on='Group', right_on='Group')
    return stat

threshold = ecod.threshold_
descriptive_stats = descriptive_stat_threshold(X_train, y_train_scores, threshold)
print(descriptive_stats)


 It is noticeable from the output that normal data points have higher average values across all features compared to outliers. The anomaly score for the outlier group is considerably higher than that of the normal group, confirming the effectiveness of the ECOD model in distinguishing between normal and anomalous observations.

### 7. Confusion Matrix

In [ ]:
# Generate synthetic dataset with ground truth labels
X_train, X_test, y_train, y_test = generate_data(
    n_train=n_train,
    n_test=n_test,
    n_features=df.shape[1],
    contamination=contamination,
    random_state=123
)
from sklearn.metrics import confusion_matrix

# Make sure the labels are in the same format
y_train = y_train.astype(int)

# Build confusion matrix
cm = confusion_matrix(y_train, y_train_pred)

print('Confusion Matrix:')
print(cm)


Finally, to evaluate the model's performance, I generated a confusion matrix for the model's predictions on the training data. 

In conclusion, the ECOD model was successful in identifying outliers in the synthetic dataset, which could represent significant anomalous events depending on the specific business context. By identifying these outliers, necessary preventative measures can be taken to address the issues they may represent. These could range from fraudulent transactions in finance to sensor malfunctions in machinery. The provided analysis and summary statistics offer a comprehensive understanding of the underlying data and the behavior of the outliers, allowing for insightful and data-driven decision-making.